In [1]:
import numpy as np
from datetime import datetime

np.random.seed(0)

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [2]:
class LogicGate:
    def __init__(self, x_data, t_data, i_node, h_node, o_node, lr, iter_count):
        if x_data.ndim == 1:    # vector
            self.x_data = x_data.reshape(1, i_node)
            self.t_data = t_data.reshape(1, o_node)
        elif x_data.ndim == 2:  # matrix
            self.x_data = x_data
            self.t_data = t_data

        self.learning_rate = lr
        self.iteration_count = iter_count
        
        self.W2 = np.random.rand(i_node, h_node)
        self.b2 = np.random.rand(1)
        
        self.W3 = np.random.rand(h_node, o_node)
        self.b3 = np.random.rand(1)
        
        print("LogicGate Object is created.\n")
        
    def __feed_forward(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))

    def __loss_val(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))
    
    def train(self):
        f = lambda x : self.__feed_forward()
        
        print("Initial loss value = ", self.__loss_val(), '\n')
        
        start_time = datetime.now()

        for step in range(self.iteration_count):
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
            
            if step % (int)(0.05*self.iteration_count) == 0:
                print("step = ", step, " : ", "loss value = ", self.__loss_val())
        
        end_time = datetime.now()

        print("")
        print("Elapsed Time => ", end_time - start_time)
        
    def predict(self, test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        if a3 >= 0.5:
            result = 1
        else:
            result = 0
    
        return a3, result
    
    def accuracy(self, test_data):
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(input_data)
        
        return accuracy_val

In [3]:
try:
    loaded_data = np.loadtxt('./diabetes.csv', delimiter=',', dtype=np.float32)

    xdata = loaded_data[:, :-1]
    tdata = loaded_data[:, -1:]
except Exception as err:
    print(str(err))

In [4]:
i_node = 8
h_node = 10
o_node = 1
lr = 1e-4
iter_count = 5001

obj1 = LogicGate(xdata, tdata, i_node, h_node, o_node, lr, iter_count)

obj1.train()

obj1.accuracy(loaded_data)

LogicGate Object is created.

Initial loss value =  349.3594928465158 

step =  0  :  loss value =  345.8993423519159
step =  250  :  loss value =  219.67294478103204
step =  500  :  loss value =  210.11814121617013
step =  750  :  loss value =  202.94021488526445
step =  1000  :  loss value =  197.2750400698293
step =  1250  :  loss value =  192.64682830848255
step =  1500  :  loss value =  188.7623899052209
step =  1750  :  loss value =  185.4260205302353
step =  2000  :  loss value =  182.50046511196786
step =  2250  :  loss value =  179.88860683072392
step =  2500  :  loss value =  177.52313345677788
step =  2750  :  loss value =  175.35913651279185
step =  3000  :  loss value =  173.36804222629257
step =  3250  :  loss value =  171.5325138164687
step =  3500  :  loss value =  169.8423021378031
step =  3750  :  loss value =  168.29104832692008
step =  4000  :  loss value =  166.87398824413455
step =  4250  :  loss value =  165.58645287761283
step =  4500  :  loss value =  164.42302

0.764163372859025